In [11]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


In [13]:
COLS_DIAGS = ["Capítulo", "Sección", "Categoría", "Descripción"]

In [14]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")
df_procesada["fecha_1_evaluacion"] = pd.to_datetime(
    df_procesada["fecha_1_evaluacion"], format="%Y-%m-%d"
)
df_procesada["f_nac"] = pd.to_datetime(df_procesada["f_nac"], yearfirst=True)

edad_primera_consulta = (
    df_procesada["fecha_1_evaluacion"] - df_procesada["f_nac"]
) / np.timedelta64(1, "Y")

df_procesada["edad_primera_consulta"] = edad_primera_consulta
df_procesada["rango_etario_primera_consulta"] = pd.cut(
    df_procesada["edad_primera_consulta"], [0, 15, 20, 25, 40, np.inf]
).astype(str)

df_procesada = df_procesada.fillna("Nulos")
df_procesada["anio_primera_evaluacion"] = df_procesada.anio_primera_evaluacion.replace(
    "Nulos", np.nan
)

# Analisis para Congenitas

En este analisis se quieren responder obtener los siguientes insights:

- Frecuencia de diagnosticos por anio
- Frecuencia de edad de primera consulta por anio (grupo etario)
- Frecuencia de procedencia por anio
- Frecuencia de sexo por anio

In [15]:
desglose_sociodemografico = ["anio_primera_evaluacion", "region"]

variables_a_analizar = [
    "diagnostico_principal",
    "prevision",
    "clasificacion",
    "complejidad",
    "rango_etario_primera_consulta",
    "sexo",
]

In [16]:
resultado_sociodemo = build_features.obtener_dfs_para_desglose_sociodemografico(
    df_procesada, desglose_sociodemografico, variables_a_analizar
)

In [17]:
print(df_procesada["diagnostico_principal"].value_counts().sum())
for var_a_analizar, df_a_analizar in resultado_sociodemo.items():
    print(f"El dataframe que analiza {var_a_analizar} tiene {df_a_analizar['conteo'].sum()} "
          f"conteos de diagnosticos")

3076
El dataframe que analiza diagnostico_principal tiene 3027 conteos de diagnosticos
El dataframe que analiza prevision tiene 3027 conteos de diagnosticos
El dataframe que analiza clasificacion tiene 3027 conteos de diagnosticos
El dataframe que analiza complejidad tiene 3027 conteos de diagnosticos
El dataframe que analiza rango_etario_primera_consulta tiene 3027 conteos de diagnosticos
El dataframe que analiza sexo tiene 3027 conteos de diagnosticos


- De los diccionarios obtenidos se puede ver que existen diferencias en las cantidades totales
de casos. Esto se debe a valores faltantes en cada una de las variables agregadas (ANIO_PRIMERA_EVALUACION,
Region, Rango Etario y Sexo)

- Esta diferencia solamente existe cuando se eliminan los valores faltantes. Si es que se convierten
a str tales valores, o se rellenan los Na (ej: con "SO"), entonces se obtiene la misma cantidad de registros.

In [18]:
with pd.ExcelWriter("../data/processed/tableros_powerbi/distribucion_sociodemografica.xlsx") as file:
    for nombre_hoja, df_a_guardar in resultado_sociodemo.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja, index=False)